In [2]:
import pandas as pd

import fnmatch
import glob
import dropbox as db

# Import Files from Dropbox Connection

In [3]:
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import dbx_token
dbx = db.Dropbox(dbx_token)

path = '/Users/matthiashugli/Dropbox/bucket/euro2020/*.csv'
# suisse_path = '/Users/matthiashugli/Dropbox/bucket/euro2020/raw-suisse-shots-against.csv'

df = pd.DataFrame()
for filename in glob.glob(path):
    #print(filename)
    file = pd.read_csv(filename, header=0, delimiter=',', low_memory=False)
    file.insert(0, 'filename', filename)
    df = df.append(file)

# suisse_df = pd.DataFrame()
# for filename in glob.glob(sommer_path):
#     #print(filename)
#     file = pd.read_csv(filename, header=1, delimiter=',', low_memory=False)
#     file.insert(0, 'filename', filename)
#     suisse_df = suisse_df.append(file)

ln: config.py: File exists


/usr/local/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
df[['source', 'file']] = df.filename.apply(lambda x: pd.Series(str(x).split('raw-')))
df[['goalkeeper', 'split6']] = df.file.apply(lambda x: pd.Series(str(x).split('.csv')))
df = df.drop(columns=['source', 'file', 'split6'])
print(df.head(5))

                                            filename        Date  Day  \
0  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-15  Tue   
1  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-19  Sat   
2  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-23  Wed   
3  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-28  Mon   
0  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-14  Mon   

         Round    Venue         Result              Squad        Opponent  \
0  Group stage  Neutral          W 1–0          fr France      de Germany   
1  Group stage  Neutral          D 1–1          fr France      hu Hungary   
2  Group stage  Neutral          D 2–2          fr France     pt Portugal   
3  Round of 16  Neutral  D 3 (4)–3 (5)          fr France  ch Switzerland   
0  Group stage  Neutral          W 2–0  cz Czech Republic    sct Scotland   

  Start Pos  ...  Att.2  Launch%.1  AvgLen.1  Opp  Stp  Stp%  #OPA  AvgDist  \
0    Y*  GK  ...   

In [8]:
xSaves = df.loc[:, ('goalkeeper', 'PSxG', 'Saves', 'SoTA', 'GA', 'PKsv')]
xSaves['shots_faced'] = xSaves['GA'] + xSaves['Saves'] + xSaves['PKsv']
xSaves['Saves%'] = ( xSaves['Saves'] + xSaves['PKsv'] ) / xSaves['shots_faced']
xSaves['xS'] = xSaves['shots_faced'] - xSaves['PSxG']
xSaves['xS%'] = xSaves['xS'] / xSaves['shots_faced']

In [15]:
gk_xSaves = xSaves.groupby('goalkeeper').agg({'GA': 'sum', 'Saves': 'sum', 'PKsv': 'sum', 'shots_faced': 'sum', 'SoTA': 'sum', 'PSxG': 'sum', 'xS': 'sum' })
gk_xSaves['xS%'] = gk_xSaves.xS / gk_xSaves.shots_faced
gk_xSaves['Saves%'] = ( gk_xSaves.Saves + gk_xSaves.PKsv ) / gk_xSaves.shots_faced
gk_xSaves['xS%2'] = gk_xSaves.xS / gk_xSaves.SoTA
gk_xSaves['Saves%2'] = gk_xSaves.Saves / gk_xSaves.SoTA
gk_xSaves['shot_performance'] = gk_xSaves['Saves%'] - gk_xSaves['xS%']
gk_xSaves['PSxG/SoTA'] = gk_xSaves['PSxG'] / gk_xSaves.SoTA
print(gk_xSaves)

            GA  Saves  PKsv  shots_faced  SoTA  PSxG    xS       xS%  \
goalkeeper                                                             
dommarumma   4      9     0           13    12   5.6   7.4  0.569231   
loris        6      7     1           14    11   5.1   8.9  0.635714   
neuer        7      4     0           11    10   4.8   6.2  0.563636   
olsen        4     11     0           15    15   4.0  11.0  0.733333   
patricio     7     10     0           17    15   5.0  12.0  0.705882   
pickford     2     14     0           16    17   2.6  13.4  0.837500   
schmeichel   7     16     1           24    21   7.2  16.8  0.700000   
sommer       9     22     0           31    30   8.9  22.1  0.712903   
vaclik       4     11     0           15    16   5.4   9.6  0.640000   

              Saves%      xS%2   Saves%2  shot_performance  PSxG/SoTA  
goalkeeper                                                             
dommarumma  0.692308  0.616667  0.750000          0.123077   0.